### InsGen

In [1]:
import json
from datasets import Dataset
import numpy as np

# 读取 JSON 文件
# InsGen
with open('/home/ma-user/work/code_dev/cth/data/output_list/ins_gen_output_list.json', 'r') as f:
    output_list = json.load(f)

print(len(output_list))

/home/ma-user/work/code_dev/miniconda3/envs/sft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


210000


In [2]:
output_list[0]

'{\n    "question1": "What factors contribute to the low efficiency of cationic lipid-mediated gene transduction in polarized and differentiated airway epithelial cells, and how might these barriers be overcome to improve gene therapy for airway diseases such as cystic fibrosis?",\n    "question2": "How do the results of gene transduction using GL-67:pDNA complexes in the nasal epithelium of cystic fibrosis transgenic mice compare to those observed in the lungs, and what implications do these differences have for the development of gene therapy strategies for cystic fibrosis?"\n}'

In [4]:
# 初始化存储成功解析的 JSON 数据和成功序号的列表
data = []
successful_indexes = []
flag = 0

# 使用列表推导解析 JSON，跳过有错误的项
for idx, item in enumerate(output_list):
    try:
        # 尝试解析 JSON
        parsed_item = json.loads(item)
        data.append(parsed_item)
        successful_indexes.append(idx)  # 保存成功解析的序号
    except json.JSONDecodeError:
        # 如果解析失败，打印错误信息并跳过当前项
        # print(item)
        flag = 1
        continue

if flag:
    print("存在解析失败")
len(output_list) - len(successful_indexes)

存在解析失败


29

In [5]:
# 使用 datasets 库创建数据集
dataset = Dataset.from_list(data)
# 输出数据集内容
print(dataset)

question1 = dataset["question1"]
question1_dict = {"question": question1}
question1_set = Dataset.from_dict(question1_dict)
print(question1_set)

question2 = dataset["question2"]
question2_dict = {"question": question2}
question2_set = Dataset.from_dict(question2_dict)
print(question2_set)

question1_set.save_to_disk("/home/ma-user/work/code_dev/cth/data/instructions/gen_ins/ins1")
question2_set.save_to_disk("/home/ma-user/work/code_dev/cth/data/instructions/gen_ins/ins2")

np.save('/home/ma-user/work/code_dev/cth/data/instructions/gen_ins/successful_indexes.npy', successful_indexes)

Dataset({
    features: ['question1', 'question2'],
    num_rows: 209971
})
Dataset({
    features: ['question'],
    num_rows: 209971
})
Dataset({
    features: ['question'],
    num_rows: 209971
})


Saving the dataset (1/1 shards): 100%|██████████| 209971/209971 [00:00<00:00, 441929.84 examples/s]


In [6]:
question2[0]

'How do the results of gene transduction using GL-67:pDNA complexes in the nasal epithelium of cystic fibrosis transgenic mice compare to those observed in the lungs, and what implications do these differences have for the development of gene therapy strategies for cystic fibrosis?'

### Scoring

In [11]:
import json
from datasets import Dataset
import numpy as np

# 读取 JSON 文件
# Scoring
# with open('/home/ma-user/work/code_dev/cth/data/output_list/scoring_1.json', 'r') as f:
#     output_list = json.load(f)
with open('/home/ubuntu/disk/output/huatuo.json', 'r') as f:
    output_list = json.load(f)

print(len(output_list))

5000


In [12]:
# 初始化存储成功解析的 JSON 数据和成功序号的列表
data = []
successful_indexes = []
flag = 0

# 使用列表推导解析 JSON，跳过有错误的项
for idx, item in enumerate(output_list):
    try:
        # 尝试解析 JSON
        parsed_item = json.loads(item)
        data.append(parsed_item)
        successful_indexes.append(idx)  # 保存成功解析的序号
    except json.JSONDecodeError:
        # 如果解析失败，打印错误信息并跳过当前项
        # print(item)
        flag = 1
        continue

if flag:
    print("存在解析失败")
len(output_list) - len(successful_indexes)

0

In [13]:
data[7]

{'quality': 7,
 'difficulty': 6,
 'relevance_to_medicine': 5,
 'mention_specific_details': 'True'}

In [13]:
# LLaMA3.1-70B need this, not for Qwen2.5-72B 
import concurrent.futures
from datasets import Dataset

# 用于存储有效和无效的记录
valid_data = []
invalid_data = []

# 预处理单条数据的函数
def process_record(index, record):
    try:
        # 如果需要，先进行数据转换
        if isinstance(record['quality'], str) and record['quality'].isdigit():
            record['quality'] = int(record['quality'])
        # 可以继续添加其他字段的检查和转换
        if isinstance(record['difficulty'], str) and record['difficulty'].isdigit():
            record['difficulty'] = int(record['difficulty'])
        if isinstance(record['relevance_to_medicine'], str) and record['relevance_to_medicine'].isdigit():
            record['relevance_to_medicine'] = int(record['relevance_to_medicine'])

        return (True, record)  # 返回有效数据
    except Exception as e:
        return (False, (index, record, str(e)))  # 返回无效数据

# 使用多进程并行处理
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_record, index, record): index for index, record in enumerate(data)}
    
    for future in concurrent.futures.as_completed(futures):
        success, result = future.result()
        if success:
            valid_data.append(result)
        else:
            invalid_data.append(result)

# 打印出无法转换的数据
if invalid_data:
    print(f"Found {len(invalid_data)} invalid records:")
    for index, record, error in invalid_data:
        print(f"Error processing record at index {index}: {record}")
        print(f"Error message: {error}")

In [14]:
# 使用 datasets 库创建数据集
ins_score_set = Dataset.from_list(data)

# 将 "ids" 列表添加到数据集中
ins_score_set = ins_score_set.add_column("ids", successful_indexes)

# 输出数据集内容
print(ins_score_set)

# ins_score_set.save_to_disk("/home/ma-user/work/code_dev/cth/data/scoring/ins1_score")
ins_score_set.save_to_disk("/home/ubuntu/disk/score/aquila")

# np.save('/home/ma-user/work/code_dev/cth/data/scoring/successful_indexes1.npy', successful_indexes)
# np.save('/home/ubuntu/disk/score/successful_indexes2.npy', successful_indexes)

Dataset({
    features: ['quality', 'difficulty', 'relevance_to_medicine', 'mention_specific_details', 'ids'],
    num_rows: 5000
})


Saving the dataset (1/1 shards): 100%|██████████| 5000/5000 [00:00<00:00, 471026.66 examples/s]


### ResGen

In [9]:
import json
from datasets import Dataset
import numpy as np

# 读取 JSON 文件
# Scoring
with open('/home/ma-user/work/code_dev/cth/data/output_list/res_output_list_common.json', 'r') as f:
    output_list = json.load(f)

# with open('/home/ma-user/work/code_dev/cth/data/output_list/res_output_list_complex1.json', 'r') as f:
#     output_list = json.load(f)
# with open('/home/ma-user/work/code_dev/cth/data/output_list/res_output_list_complex2.json', 'r') as f:
#     output_list = json.load(f)
print(len(output_list), output_list[0])

500 {
    "answer1": "Financial penalties for non-compliance by healthcare entities can have significant impacts on the quality and accessibility of healthcare services. These penalties, often imposed for violations of regulatory standards such as patient privacy (HIPAA) or quality of care, can lead to financial strain on healthcare facilities. This strain can result in reduced resources for patient care, fewer staff, and less investment in essential medical equipment and training, which can directly affect the quality of care provided. For under-resourced facilities, these penalties can be particularly detrimental, as they often have limited financial reserves to begin with. To ensure these penalties do not disproportionately affect under-resourced facilities, several measures can be taken. Firstly, regulatory bodies can provide technical assistance and training to help these facilities understand and comply with regulations. Secondly, a graduated penalty system can be implemented, wh

In [10]:
# common 解析, comlex 不解析
# 初始化存储成功解析的 JSON 数据和成功序号的列表
data = []
successful_indexes = []
flag = 0

# 使用列表推导解析 JSON，跳过有错误的项
for idx, item in enumerate(output_list):
    try:
        # 尝试解析 JSON
        parsed_item = json.loads(item)
        data.append(parsed_item)
        successful_indexes.append(idx)  # 保存成功解析的序号
    except json.JSONDecodeError:
        # 如果解析失败，打印错误信息并跳过当前项
        flag = 1
        continue

if flag:
    print("存在解析失败")
len(output_list) - len(successful_indexes)

0

In [27]:
# common:
# 使用 datasets 库创建数据集
dataset = Dataset.from_list(data)
# 输出数据集内容
print(dataset)

answer1 = dataset["answer1"]
answer1_dict = {"answer": answer1}
answer1_set = Dataset.from_dict(answer1_dict)
print(answer1_set)

answer2 = dataset["answer2"]
answer2_dict = {"answer": answer2}
answer2_set = Dataset.from_dict(answer2_dict)
print(answer2_set)

answer1_set.save_to_disk("/home/ubuntu/disk/gen_response/res1")
answer2_set.save_to_disk("/home/ubuntu/disk/gen_response/res2")

np.save('/home/ubuntu/disk/gen_response/successful_indexes.npy', successful_indexes)

Dataset({
    features: ['answer1', 'answer2'],
    num_rows: 95
})
Dataset({
    features: ['answer'],
    num_rows: 95
})
Dataset({
    features: ['answer'],
    num_rows: 95
})


Saving the dataset (1/1 shards): 100%|██████████| 95/95 [00:00<00:00, 10861.63 examples/s]


In [31]:
# complex: 
answer_dict = {"answer": output_list}
answer_set = Dataset.from_dict(answer_dict)
print(answer_set)

# answer_set.save_to_disk("/home/ubuntu/test_disk/res1_qwq")
answer_set.save_to_disk("/home/ubuntu/test_disk/res2_qwq")

Dataset({
    features: ['answer'],
    num_rows: 1
})


Saving the dataset (1/1 shards): 100%|██████████| 1/1 [00:00<00:00, 177.04 examples/s]
